In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [33]:
import numpy as np
import os, sys
sys.path.append('..')
from baseline import *
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.utils import resample
from sklearn import linear_model

from tqdm import tqdm_notebook as tqdm

In [ ]:
sectors = [f'sector-{i}' for i in np.arange(1,23)]
rawData, y =loadData(sectors, '../')

In [ ]:
np.sum(y), len(y)-np.sum(y)

In [5]:
# Features:
  #   [0] astronet score
  #   [1] depth best ap - 1
  #   [2] depth best ap (global)
  #   [3] depth best ap + 1 or best ap
  #   [4] error best ap - 1
  #   [5] error best ap (global)
  #   [6] error best ap + 1
  # 
  #   [7] depth best ap - 1
  #   [8] depth best ap (local)
  #   [9] depth best ap + 1 or best ap
  #   [10] error best ap - 1
  #   [11] error best ap (local)
  #   [12] error best ap + 1
  # 
  #   [13] depth even
  #   [14] depth odd
  #   [15] error even
  #   [16] error odd

  #   [17] depth secondary
  #   [18] error seconday

In [25]:
x_1 = rawData[:,[0,7,8,9,13,14,17]]
x_2 = rawData[:,[0,1,2,3,7,8,9,13,14,17]]
x_3 = rawData[:,[0,1,2,3,7,8,9,13,14,15,16,17,18]]
x_full = rawData

In [24]:
print('Random Forest')
RFresults, RFimportances = testModel(x,y,RFModel, 10)
RFimportances=RFimportances[0]
print(f'{int(np.sum(y))} - True Positive, {int(np.sum(1-y))} - True Negative')
print('TN, TP, FN, FP')
for kfold in RFresults:
  print(kfold)
print(np.sum(RFresults,0))
print('Feature Importances')
print(np.median(RFimportances,0))

Random Forest
1 / 5 2 / 5 3 / 5 4 / 5 5 / 5 782 - True Positive, 131481 - True Negative
TN, TP, FN, FP
[23323   114    34  2982]
[23455   122    35  2841]
[23631   137    38  2647]
[23423   113    37  2879]
[23518   121    31  2782]
[117350    607    175  14131]
Feature Importances
[0.10982902 0.12294402 0.13231255 0.21628334 0.11037802 0.08152227
 0.22673078]


In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state = 420)

In [45]:
x_train, y_train = overSamplePos(x_train,y_train)
clf = RandomForestClassifier(n_estimators=100,max_depth=2)
clf.fit(x_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
clf.score(x_test,y_test)

0.7939272969213089

In [30]:
print(f'{int(np.sum(y))} - True Positive, {int(np.sum(1-y))} - True Negative')
for i,x in enumerate([x_1,x_2,x_3,x_full]):
  print(['Depths, No Global','Depths Only', 'Error on EOS', 'full'][i])
  print('--------')
#   print('Linear Regression')
  LRresults, LRcoefs = testModel(x,y,LRModel, 0.05)
  LRcoefs=LRcoefs[0]
  print('TN, TP, FN, FP')
#   for kfold in LRresults:
#     print(kfold)
  print(np.sum(LRresults,0))
  print('Coefficients')
  print(np.median(LRcoefs,0))
  
  print('')
  print('')
  print('------------------------------')
  print('')
  print('')

782 - True Positive, 131481 - True Negative
Depths, No Global
--------
1 / 5 2 / 5 3 / 5 4 / 5 5 / 5 TN, TP, FN, FP
[36842   740    42 94639]
Coefficients
[ 0.28878379 -0.0730868   0.31740804  1.133522    0.05700832  0.17759356
 -0.22604455]


------------------------------


Depths Only
--------
1 / 5 2 / 5 3 / 5 4 / 5 5 / 5 TN, TP, FN, FP
[36675   741    41 94806]
Coefficients
[ 0.29071973  0.12308021 -0.08781888  0.44291226 -0.18052222  0.3298281
  0.84269341  0.05090479  0.13638939 -0.27144229]


------------------------------


Error on EOS
--------
1 / 5 2 / 5 3 / 5 4 / 5 5 / 5 TN, TP, FN, FP
[41034   732    50 90447]
Coefficients
[ 0.27514413  0.13320003 -0.12619403  0.46977216 -0.15773474  0.12734443
  0.87113433  0.04755169  0.17790745 -1.20252541  0.15022767 -0.39506614
 -2.42896775]


------------------------------


full
--------
1 / 5 2 / 5 3 / 5 4 / 5 5 / 5 TN, TP, FN, FP
[41936   733    49 89545]
Coefficients
[ 0.27059689  0.15783336 -0.15827252  0.41417607  1.27384525 -